# 2. 파이썬으로 데이터 주무르기, pandas
**pandas를 활용해서 데이터프레임을 다뤄봅시다.**

1. Pandas 시작하기
    - prerequisite : Table
    - pandas import하기
   
2. Pandas로 1차원 데이터 다루기 - Series 
    - Series 선언하기
    - Series vs ndarray
    - Series vs dict
    - Series에 이름 붙이기
3. Pandas로 2차원 데이터 다루기 - dataframe
    - dataframe 선언하기
    - from csv to dataframe
    - dataframe 자료 접근하기

[수업에 사용된 covid 데이터](https://www.kaggle.com/imdevskp/corona-virus-report)

## I. pandas 시작하기

## II. pandas로 1차원 데이터 다루기 - Series

## III. Pandas로 2차원 데이터 다루기 - dataframe

## Mission:
### 1. covid 데이터에서 100 case 대비 사망률(`Deaths / 100 Cases`)이 가장 높은 국가는?

In [19]:
import pandas as pd 
data = pd.read_csv('country_wise_latest.csv')
idx_max = data[data.columns[10]].idxmax()
print(data[data.columns[0]][idx_max])

Canada


### 2. covid 데이터에서 신규 확진자가 없는 나라 중 WHO Region이 'Europe'를 모두 출력하면?  
Hint : 한 줄에 동시에 두가지 조건을 Apply하는 경우 Warning이 발생할 수 있습니다.

In [38]:
first_cond = data[data[data.columns[5]] == 0]
two_conds = first_cond[first_cond[data.columns[-1]] == 'Europe']
print(two_conds.iloc[:, 0].to_list())

['Estonia', 'Holy See', 'Latvia', 'Liechtenstein', 'Monaco', 'San Marino', 'Spain']


### 3. 다음 [데이터](https://www.kaggle.com/neuromusic/avocado-prices)를 이용해 각 Region별로 아보카도가 가장 비싼 평균가격(AveragePrice)을 출력하면?

In [58]:
avo = pd.read_csv('avocado.csv')
regions = avo['region'].unique()
avg_price = []
for r in regions:
    avo_region = avo[avo['region'] == r]
    avg_price.append(avo_region['AveragePrice'].max())
result = pd.DataFrame({"AveragePrice": avg_price, "region": regions})
result.head()

,AveragePrice,region
0,2.13,Albany
1,2.75,Atlanta
2,2.28,BaltimoreWashington
3,2.79,Boise
4,2.19,Boston
